# Optimizing warehouse flows with Q-learning ###
________

In [157]:
import numpy as np
import random
from typing import List

## The problem

![](case1.png)

## Defining the enviorment
_____________________________________


#### States
The set of environmental states S is defined as the finite set {s1, . . . , sN } where the size of the state space is N, i.e. |S| = N.

- location at time t

In [158]:
locations_encoding = {letter:i for i, letter in enumerate( "ABCDEFGHIJKL")} #locations_encoding
print("locations mapping: ", locations_encoding)

states = set(locations_encoding.values())
print("\n\nstates: ", states)

locations mapping:  {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11}


states:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}


#### Actions
The set of actions A is defined as the finite set {a1, . . . , aK} where the size of the action space is K, i.e. |A| = K. Actions can be used to control the system state. The set of actions that can be applied in some particular state s ∈ S, is denoted A(s), where A(s) ⊆ A.
- set of neighbours of given location

In [159]:
neighbors = {letter:[] for letter in "ABCDEFGHIJKL"} 

# populating neighbours
neighbors["A"].append("B")
neighbors["B"].extend(["A", "F", "C"])
neighbors["C"].extend(["B", "G"])
neighbors["D"].append("H")
neighbors["E"].append("I")
neighbors["F"].extend(["B", "J"])
neighbors["G"].extend(["C", "H"])
neighbors["H"].extend(["D", "G", "L"])
neighbors["I"].extend(["E", "J"])
neighbors["J"].extend(["F", "I", "K"])
neighbors["K"].extend(["J", "L"])
neighbors["L"].extend(["K", "H"])
print("neighbors dictionary: ", neighbors)

actions = [set(neighbors[letter]) for letter in locations_encoding.keys()]
print("\nactions with letters: ", actions)
actions = [set(locations_encoding[i] for i in neighbors[letter]) for letter in locations_encoding.keys()]
print("\nactions: ", actions)

neighbors dictionary:  {'A': ['B'], 'B': ['A', 'F', 'C'], 'C': ['B', 'G'], 'D': ['H'], 'E': ['I'], 'F': ['B', 'J'], 'G': ['C', 'H'], 'H': ['D', 'G', 'L'], 'I': ['E', 'J'], 'J': ['F', 'I', 'K'], 'K': ['J', 'L'], 'L': ['K', 'H']}

actions with letters:  [{'B'}, {'C', 'A', 'F'}, {'B', 'G'}, {'H'}, {'I'}, {'B', 'J'}, {'C', 'H'}, {'L', 'G', 'D'}, {'E', 'J'}, {'K', 'I', 'F'}, {'L', 'J'}, {'K', 'H'}]

actions:  [{1}, {0, 2, 5}, {1, 6}, {7}, {8}, {1, 9}, {2, 7}, {11, 3, 6}, {9, 4}, {8, 10, 5}, {9, 11}, {10, 7}]


#### Rewards

- 2D matrix of position and move

In [160]:
rewards = np.array(np.zeros([len(locations_encoding), len(locations_encoding)])) # initially 0 (move not possible)

for i, location in enumerate(locations_encoding.keys()):
    for neighbor in neighbors[location]:
        rewards[i][locations_encoding[neighbor]] = 1

# as we want to train algorithm to go to G, we will write 1000 at (G,G)
rewards[locations_encoding["G"]][locations_encoding["G"]] = 1000

print("rewards matrix: ", rewards)



rewards matrix:  [[   0.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [   1.    0.    1.    0.    0.    1.    0.    0.    0.    0.    0.    0.]
 [   0.    1.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.]
 [   0.    1.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.]
 [   0.    0.    1.    0.    0.    0. 1000.    1.    0.    0.    0.    0.]
 [   0.    0.    0.    1.    0.    0.    1.    0.    0.    0.    0.    1.]
 [   0.    0.    0.    0.    1.    0.    0.    0.    0.    1.    0.    0.]
 [   0.    0.    0.    0.    0.    1.    0.    0.    1.    0.    1.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    1.]
 [   0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    1.    0.]]


## Q-learning solution
_____________________________________


![](case1-pseudo.png)

In [161]:
class QLearningSolver:
    def __init__(self, states: set, actions: List[set], start_state:int, end_state:int, repetitions=1000, discount_factor = 0.75, learning_rate=0.9):
        self.states = states
        self.actions = actions
        self.repetitions = repetitions
        self.gamma = discount_factor
        self.lr = learning_rate
        self.start = start_state
        self.end = end_state
        self.rewards = self.get_rewards()
        self.qvalues = self.get_qvalues()
    
    def select_random_state(self):
        return random.choice(tuple(self.states))
    
    def perform_random_action(self, curr: int):
        return random.choice(tuple(self.actions[curr]))
    
    def get_rewards(self):
        rewards = np.array(np.zeros([len(self.states), len(self.actions)]))
        for state in self.states:
            for next_state in self.actions[state]:
                rewards[state][next_state] = 1
        rewards[self.end][self.end] = 1000
        self.actions[self.end].add(self.end)
        return rewards
            
    def get_qvalues(self):
        qvalues = np.array(np.zeros([len(self.states), len(self.actions)]))
        for _ in range(self.repetitions):
            # 1. Select random state
            curr_state = self.select_random_state()
            # 2. Play random action that can lead to next possible state
            next_state = self.perform_random_action(curr_state)
            # 3. We reach new state and get the reward
            reward = self.rewards[curr_state][next_state]
            # 4. We compute the Temporal Difference TD
            td = reward + self.gamma * max([qvalues[next_state][a2] for a2 in self.actions[next_state]]) - qvalues[curr_state][next_state]
            # 5. We update Q-value by applying the Bellmann equation
            qvalues[curr_state][next_state] = qvalues[curr_state][next_state] + self.lr * td
        return qvalues
    
    def get_route(self):
        route = [self.start]
        curr_state = self.start
        while curr_state != self.end:
            curr_state = np.argmax(self.qvalues[curr_state, ])
            route.append(curr_state)
        return route

In [162]:
states = set(locations_encoding.values())
actions = [set(locations_encoding[i] for i in neighbors[letter]) for letter in locations_encoding.keys()]
solver = QLearningSolver(states, actions, locations_encoding["E"], locations_encoding["G"])

print("Rewards:\n\n", solver.rewards.astype(int))
print("\n\nQ-valules:\n\n", solver.qvalues.astype(int))

Rewards:

 [[   0    1    0    0    0    0    0    0    0    0    0    0]
 [   1    0    1    0    0    1    0    0    0    0    0    0]
 [   0    1    0    0    0    0    1    0    0    0    0    0]
 [   0    0    0    0    0    0    0    1    0    0    0    0]
 [   0    0    0    0    0    0    0    0    1    0    0    0]
 [   0    1    0    0    0    0    0    0    0    1    0    0]
 [   0    0    1    0    0    0 1000    1    0    0    0    0]
 [   0    0    0    1    0    0    1    0    0    0    0    1]
 [   0    0    0    0    1    0    0    0    0    1    0    0]
 [   0    0    0    0    0    1    0    0    1    0    1    0]
 [   0    0    0    0    0    0    0    0    0    1    0    1]
 [   0    0    0    0    0    0    0    1    0    0    1    0]]


Q-valules:

 [[   0 1685    0    0    0    0    0    0    0    0    0    0]
 [1262    0 2246    0    0 1265    0    0    0    0    0    0]
 [   0 1685    0    0    0    0 2993    0    0    0    0    0]
 [   0    0    0    0    0  

## Going into production
_____________________________________


In [191]:
solver = QLearningSolver(states, actions, locations_encoding["E"], locations_encoding["G"])
states_mapping = {val:key for key, val in locations_encoding.items()}
route = solver.get_route()
print([states_mapping[state] for state in route])
# ['E', 'I', 'J', 'F', 'B', 'C', 'G'] or ['E', 'I', 'J', 'K', 'L', 'H', 'G']

['E', 'I', 'J', 'K', 'L', 'H', 'G']


## Intermediary location
_____________________________________

#### Method 3 solution (quick & better):

In [200]:
route = QLearningSolver(states, actions, locations_encoding["E"], locations_encoding["K"]).get_route() + QLearningSolver(states, actions, locations_encoding["K"], locations_encoding["G"]).get_route()
del route[int(len(route)/2)] # remove repeted letter
print([states_mapping[state] for state in route])

['E', 'I', 'J', 'K', 'L', 'H', 'G']


____
#### Other solution (worse):

In [164]:
class QLearningSolverWithIntermediary(QLearningSolver):
    def __init__(self, states: set, actions: List[set], start_state: int, end_state: int, intermediary_from: int, intermediary_to: int, repetitions=1000, discount_factor=0.75, learning_rate=0.9):
        self.inter_start = intermediary_from
        self.inter_end = intermediary_to
        super().__init__(states, actions, start_state, end_state, repetitions, discount_factor, learning_rate)

    def get_rewards(self):
        super().get_rewards()
        rewards[self.inter_start][self.inter_end] = 500
        return rewards
    

In [165]:
states = set(locations_encoding.values())
actions = [set(locations_encoding[i] for i in neighbors[letter]) for letter in locations_encoding.keys()]
solver2 = QLearningSolverWithIntermediary(states, actions, locations_encoding["E"], locations_encoding["G"], locations_encoding["J"], locations_encoding["K"])

print("Rewards:\n\n", solver2.rewards.astype(int))
print("\n\nQ-valules:\n\n", solver2.qvalues.astype(int))

Rewards:

 [[   0    1    0    0    0    0    0    0    0    0    0    0]
 [   1    0    1    0    0    1    0    0    0    0    0    0]
 [   0    1    0    0    0    0    1    0    0    0    0    0]
 [   0    0    0    0    0    0    0    1    0    0    0    0]
 [   0    0    0    0    0    0    0    0    1    0    0    0]
 [   0    1    0    0    0    0    0    0    0    1    0    0]
 [   0    0    1    0    0    0 1000    1    0    0    0    0]
 [   0    0    0    1    0    0    1    0    0    0    0    1]
 [   0    0    0    0    1    0    0    0    0    1    0    0]
 [   0    0    0    0    0    1    0    0    1    0  500    0]
 [   0    0    0    0    0    0    0    0    0    1    0    1]
 [   0    0    0    0    0    0    0    1    0    0    1    0]]


Q-valules:

 [[   0 1689    0    0    0    0    0    0    0    0    0    0]
 [1267    0 2251    0    0 1266    0    0    0    0    0    0]
 [   0 1689    0    0    0    0 3000    0    0    0    0    0]
 [   0    0    0    0    0  

In [187]:
solver2 = QLearningSolverWithIntermediary(states, actions, locations_encoding["E"], locations_encoding["G"], locations_encoding["J"], locations_encoding["K"])
route = solver2.get_route()
print([states_mapping[state] for state in route]) # note: always the same route through K: ['E', 'I', 'J', 'K', 'L', 'H', 'G']

['E', 'I', 'J', 'K', 'L', 'H', 'G']


## References
____

1. Markov Decision Processes: Concepts and Algorithms, Martijn van Otterlo, May 2009

2. Artificial Intelligence for Business Udemy Course  [Link to the course](https://www.udemy.com/share/101Y7E3@ufylekA5UWd1-mxWKBL2ecxPn01pSdl5tfu-wGkkbyEcOpnoF9HyJXi_3vey6XqgmQ==/)
